In [42]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
from tqdm import tqdm
import os
from chinese_calendar import is_holiday, is_workday
import datetime
import geopandas
import networkx as nx  
from shapely.geometry import Point, LineString
from geodatasets import get_path 
import folium
from folium import plugins
from branca.colormap import LinearColormap

In [2]:
stationInfo = pd.read_csv('../../Metro/station_transInfo_cleaned.csv')
dict_staionName = dict(zip(stationInfo['stationID'], stationInfo['name']))
dict_stationLoc = dict(zip(stationInfo['stationID'], zip(stationInfo['lon'],stationInfo['lat'])))
stationID_List = list(stationInfo['stationID'])

In [9]:
def RushHourConnection(targetStationID, stationID_List): # Flow Mon-Fri

    morningRushCount = [0] * len(stationID_List) # Morning Orgin
    eveningRushCount = [0] * len(stationID_List) # evening destination
    currDate = '20170501'
    isHoliday = True

    ODFile = open('/data6/peiyan/SH-METR/results/metroData_ODFlow_10.csv','r')
    next(ODFile)

    for row in ODFile: 
        row = row.rstrip().split(',')
        if row[0] != currDate:
            print(currDate)
            currDate = row[0]
            isHoliday = is_holiday(datetime.datetime.strptime(currDate,'%Y%m%d'))
        
        if isHoliday:
            continue
        
        if int(row[4]) == targetStationID: # evening
            hour = int(row[2])//10000
            if hour >16 and hour <19:
                destinationStationID = int(row[5])
                eveningRushCount[stationID_List.index(destinationStationID)] += int(row[6])
        
        if int(row[5]) == targetStationID: # moring
            hour = int(row[2])//10000
            if hour >6 and hour <9:
                originStationID = int(row[4])
                morningRushCount[stationID_List.index(originStationID)] += int(row[6])

    return morningRushCount, eveningRushCount

In [10]:
# 对目标站点, 早高峰 top10 的通勤 Trip 来源； 晚高峰 top10 的通勤 Trip 去向  是否吻合
targetStationID = 247 # 陆家嘴

morningRushCount, eveningRushCount = RushHourConnection(targetStationID, stationID_List)

20170501
20170502
20170503
20170504
20170505
20170506
20170507
20170508
20170509
20170510
20170511
20170512
20170513
20170514
20170515
20170516
20170517
20170518
20170519
20170520
20170521
20170522
20170523
20170524
20170525
20170526
20170527
20170528
20170529
20170530
20170531
20170601
20170602
20170603
20170604
20170605
20170606
20170607
20170608
20170609
20170610
20170611
20170612
20170613
20170614
20170615
20170616
20170617
20170618
20170619
20170620
20170621
20170622
20170623
20170624
20170625
20170626
20170627
20170628
20170629
20170630
20170701
20170702
20170703
20170704
20170705
20170706
20170707
20170708
20170709
20170710
20170711
20170712
20170713
20170714
20170715
20170716
20170717
20170718
20170719
20170720
20170721
20170722
20170723
20170724
20170725
20170726
20170727
20170728
20170729
20170730
20170731
20170801
20170802
20170803
20170804
20170805
20170806
20170807
20170808
20170809
20170810
20170811
20170812
20170813
20170814
20170815
20170816
20170817
20170818
20170819
2

In [11]:
morningRushCount = np.array(morningRushCount)
eveningRushCount = np.array(eveningRushCount)

indicesMRC = np.argsort(morningRushCount)[::-1]
indicesERC = np.argsort(eveningRushCount)[::-1]

TopTen_MRC_station = [dict_staionName[stationID_List[idx]] for idx in indicesMRC[:10]]
TopTen_ERC_station = [dict_staionName[stationID_List[idx]] for idx in indicesERC[:10]]

print(TopTen_MRC_station)
print(TopTen_ERC_station)


['新闸路', '创新中路', '徐泾东', '长江南路', '殷高西路', '九亭', '漕溪路', '花桥', '浦东国际机场', '复兴岛']
['创新中路', '新闸路', '殷高西路', '徐泾东', '浦东国际机场', '长江南路', '中春路', '九亭', '复兴岛', '泗泾']


In [53]:
def displayOnMap(RushCount , TopTen_station_idx, stationID_List, dict_stationLoc, stationInfo, mode):

    map_center = (np.mean(stationInfo["lat"]), np.mean(stationInfo["lon"]))  
    m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB darkmatter')

    # draw all the samples: larger counts -> bigger shape, warmer color 
    colormap = LinearColormap(colors=['blue','white', 'red'],vmin= 0, vmax=np.max(RushCount))

   
    for nodeID, loc in dict_stationLoc.items(): 
        loc = (loc[1], loc[0])
        node_name = dict_staionName[nodeID]
        flowCount = RushCount[stationID_List.index(nodeID)]
        radius = flowCount/np.max(RushCount)*30
        if node_name == '陆家嘴':
            continue
        folium.CircleMarker(location=loc, popup=node_name, radius=radius, color=colormap(flowCount), fillOpacity=0.3, fill=True, stroke=False).add_to(m)

    centerPos = (dict_stationLoc[247][1], dict_stationLoc[247][0])
    folium.Marker(location=centerPos,popup="陆家嘴", icon=folium.Icon(color="yellow", icon="star", prefix='fa')).add_to(m)

    for idx in TopTen_station_idx:
        ID = stationID_List[idx]
        if mode =='moring':
            startPos = (dict_stationLoc[ID][1], dict_stationLoc[ID][0])
            endPos =  centerPos
        else: 
            endPos = (dict_stationLoc[ID][1], dict_stationLoc[ID][0])
            startPos = centerPos

        folium.plugins.AntPath(locations=[startPos, endPos],weight=2,color='green',opacity=0.8, delay=1000).add_to(m)


    return m

In [55]:
displayOnMap(morningRushCount, indicesMRC[:10], stationID_List, dict_stationLoc, stationInfo, 'morning')

/tmp/ipykernel_38760/3258548236.py:20: UserWarning: color argument of Icon should be one of: {'blue', 'darkblue', 'darkgreen', 'lightblue', 'orange', 'cadetblue', 'darkpurple', 'green', 'pink', 'black', 'red', 'lightred', 'darkred', 'purple', 'lightgreen', 'white', 'gray', 'beige', 'lightgray'}.
  folium.Marker(location=centerPos,popup="陆家嘴", icon=folium.Icon(color="yellow", icon="star", prefix='fa')).add_to(m)


In [56]:
displayOnMap(eveningRushCount, indicesERC[:10], stationID_List, dict_stationLoc, stationInfo, 'evening')

/tmp/ipykernel_38760/3258548236.py:20: UserWarning: color argument of Icon should be one of: {'blue', 'darkblue', 'darkgreen', 'lightblue', 'orange', 'cadetblue', 'darkpurple', 'green', 'pink', 'black', 'red', 'lightred', 'darkred', 'purple', 'lightgreen', 'white', 'gray', 'beige', 'lightgray'}.
  folium.Marker(location=centerPos,popup="陆家嘴", icon=folium.Icon(color="yellow", icon="star", prefix='fa')).add_to(m)
